In [76]:
import pandas as pd
from scapy.all import rdpcap
import plotly.graph_objects as go
import numpy as np
from decimal import Decimal

In [77]:
PCAP_FOLDER = "/home/sebastfu/effect_tcpreplay"
NR = 3

In [78]:
def pcap_to_dataframe(pcap_file):
    """
    Function to read packets from a pcap file and convert them into a DataFrame.
    """
    packets = rdpcap(pcap_file)
    
    # Initialize lists to store data
    timestamps = []
    src_ips = []
    dst_ips = []
    
    # Extract timestamp, source IP, and destination IP from each packet
    for packet in packets:
        timestamps.append(packet.time)
        src_ips.append(packet[1].src)  # Assuming IP is at layer 1 (index 1)
        dst_ips.append(packet[1].dst)  # Assuming IP is at layer 1 (index 1)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Timestamp': timestamps,
        'SrcIP': src_ips,
        'DstIP': dst_ips
    })

    df['iat'] = df['Timestamp'].diff()
    
    return df


def getDurationOfPcap(df):
    return (df["Timestamp"].max() - df["Timestamp"].min())

def getAmountOfPackets(df):
    return len(df.index)

In [79]:
def compareDFs(df1, df2):
    # Function to convert Decimal to float
    def decimal_to_float(val):
        if isinstance(val, Decimal):
            return float(val)
        return val
    
    # Convert DataFrame values to numeric types
    df1 = df1.applymap(decimal_to_float)
    df2 = df2.applymap(decimal_to_float)
    
    comparisonStats = {}  # Initialize with an empty dictionary

    comparisonStats["duration_diff"] = abs(getDurationOfPcap(df1) - getDurationOfPcap(df2))
    comparisonStats["packet_amount_diff"] = abs(getAmountOfPackets(df1) - getAmountOfPackets(df2))
    comparisonStats["mu_diff"] = abs(df1["iat"].mean() - df2["iat"].mean())
    comparisonStats["sigma_diff"] = abs(df1["iat"].std() - df2["iat"].std())
    comparisonStats["difference_99_percentile"] = (df1["iat"] - df2["iat"]).quantile(0.99)

    # Convert dictionary to DataFrame
    comparisonStats_df = pd.DataFrame([comparisonStats])

    return comparisonStats_df



In [80]:
originalDF = pcap_to_dataframe(f"{PCAP_FOLDER}/replay_{NR}.pcapng")
replayDF= pcap_to_dataframe(f"{PCAP_FOLDER}/tcpreplay_validation_capture_{NR}.pcapng")

In [81]:
comparision = compareDFs(originalDF, replayDF)
comparision.head()

/tmp/ipykernel_203842/1063766553.py:9: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_203842/1063766553.py:10: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,duration_diff,packet_amount_diff,mu_diff,sigma_diff,difference_99_percentile
0,0.000017,0,1.715572e-09,7.199469e-08,0.000001


In [82]:
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n+1) / n
    return x, y

x1, y1 = ecdf(originalDF['iat'].dropna())
x2, y2 = ecdf(replayDF['iat'].dropna())

fig = go.Figure()

# Add trace for the first DataFrame
fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', name='Iat in the generated pcap'))

# Add trace for the second DataFrame
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name='Iat in the replayed pcap'))

# Update layout
fig.update_layout(title='ECDF Plot',
                  xaxis_title='Value',
                  yaxis_title='ECDF')

# Show the plot
fig.show()